In [11]:
from pysmt.shortcuts import *
from pysmt.typing import INT, REAL
import math

# ============================================
# PARTE 1: MODELAGEM COM DIFERENCIAÇÃO POR SETOR
# ============================================

# Constantes do sistema
NUM_SETORES = 15
LADO_SETOR = 1.0  # 1 km
VEL_MAX = 5.0  # velocidade máxima global em m/s
SIGMA = 0.1  # coeficiente de atrito

# Modos dos navios
NAVIO_PARADO = Int(0)
NAVIO_MOVENDO = Int(1)

# Sinais do semáforo
SINAL_VERMELHO = Int(0)
SINAL_AMARELO = Int(1)
SINAL_VERDE = Int(2)

class Navio:
    """Classe que representa um navio como autômato híbrido - COM PARÂMETROS POR SETOR"""
    
    def __init__(self, nome, setor_inicial, direcao):
        self.nome = nome
        self.setor_inicial = setor_inicial
        self.direcao = direcao  # 1 para A->B, -1 para B->A
        
        # Rumo fixo baseado na direção (em radianos)
        self.rumo = 0.0 if direcao == 1 else math.pi
        
        # PARÂMETROS POR SETOR conforme especificado no enunciado
        # Cada setor tem: (γ, ε, V_local, tipo_zona)
        # γ: aceleração linear (propulsão)
        # ε: força da corrente
        # V_local: limite de velocidade no setor
        # tipo_zona: 0=aceleração, 1=desaceleração, 2=velocidade constante, 3=velocidade baixa
        
        self.parametros_setor = {}
        
        # Mapeamento de índices conforme descrito no enunciado
        # Na rota A→B:
        # - Zonas de aceleração (γ ≈ 1): {s11, s13} e {s2, s4}
        # - Zonas de desaceleração (γ ≈ 0): {s1, s5} e {s12, s14}
        # - Zonas de velocidade constante: {s3, s6, s7, s8, s9, s10}
        # - Zona de velocidade baixa: {s0}
        
        for i in range(NUM_SETORES):
            if direcao == 1:  # Rota A->B
                if i in [11, 13, 2, 4]:  # Zonas de aceleração (γ ≈ 1)
                    self.parametros_setor[i] = {
                        'gamma': 1.0,  # γ ≈ 1
                        'epsilon': 0.05,  # corrente favorável
                        'V_local': VEL_MAX,  # velocidade máxima permitida
                        'tipo': 'aceleracao'
                    }
                elif i in [1, 5, 12, 14]:  # Zonas de desaceleração (γ ≈ 0)
                    self.parametros_setor[i] = {
                        'gamma': -0.5,  # γ ≈ -0.5 (desaceleração)
                        'epsilon': -0.02,  # corrente contra
                        'V_local': VEL_MAX,
                        'tipo': 'desaceleracao'
                    }
                elif i in [3, 6, 7, 8, 9, 10]:  # Zonas de velocidade constante
                    self.parametros_setor[i] = {
                        'gamma': 0.0,  # γ = 0 (velocidade constante)
                        'epsilon': 0.0,  # sem corrente
                        'V_local': VEL_MAX,
                        'tipo': 'constante'
                    }
                elif i == 0:  # Zona de velocidade baixa
                    self.parametros_setor[i] = {
                        'gamma': 0.0,
                        'epsilon': 0.0,
                        'V_local': 2.0,  # velocidade reduzida
                        'tipo': 'baixa'
                    }
                else:
                    # Setores não especificados (por segurança)
                    self.parametros_setor[i] = {
                        'gamma': 0.0,
                        'epsilon': 0.0,
                        'V_local': VEL_MAX,
                        'tipo': 'default'
                    }
            else:  # Rota B->A (trocado conforme enunciado)
                # Zonas de aceleração e desaceleração trocadas em relação a A->B
                if i in [1, 5, 12, 14]:  # Zonas de aceleração (γ ≈ 1) - TROCADO
                    self.parametros_setor[i] = {
                        'gamma': 1.0,
                        'epsilon': 0.05,
                        'V_local': VEL_MAX,
                        'tipo': 'aceleracao'
                    }
                elif i in [11, 13, 2, 4]:  # Zonas de desaceleração (γ ≈ 0) - TROCADO
                    self.parametros_setor[i] = {
                        'gamma': -0.5,
                        'epsilon': -0.02,
                        'V_local': VEL_MAX,
                        'tipo': 'desaceleracao'
                    }
                elif i in [3, 6, 7, 8, 9, 10]:  # Zonas de velocidade constante
                    self.parametros_setor[i] = {
                        'gamma': 0.0,
                        'epsilon': 0.0,
                        'V_local': VEL_MAX,
                        'tipo': 'constante'
                    }
                elif i == 14:  # Zona de velocidade baixa (equivalente a s0 na rota oposta)
                    self.parametros_setor[i] = {
                        'gamma': 0.0,
                        'epsilon': 0.0,
                        'V_local': 2.0,
                        'tipo': 'baixa'
                    }
                else:
                    self.parametros_setor[i] = {
                        'gamma': 0.0,
                        'epsilon': 0.0,
                        'V_local': VEL_MAX,
                        'tipo': 'default'
                    }
    
    def get_parametros(self, setor):
        """Retorna os parâmetros (γ, ε, V_local) para um setor"""
        params = self.parametros_setor.get(setor)
        if params:
            return (params['gamma'], params['epsilon'], params['V_local'])
        return (0.0, 0.0, VEL_MAX)
    
    def get_tipo_zona(self, setor):
        """Retorna o tipo de zona do setor"""
        params = self.parametros_setor.get(setor)
        if params:
            return params['tipo']
        return 'default'
    
    def proximo_setor_expr(self, setor_var):
        """Retorna uma expressão para o próximo setor baseado na direção"""
        if self.direcao == 1:  # A->B
            return Ite(
                LT(setor_var, Int(NUM_SETORES - 1)),
                Plus(setor_var, Int(1)),
                setor_var  # fica no último setor
            )
        else:  # B->A
            return Ite(
                GT(setor_var, Int(0)),
                Minus(setor_var, Int(1)),
                setor_var  # fica no primeiro setor
            )
    
    def get_ponto_inicial_geometrico(self, setor):
        """Retorna ponto inicial (x0, y0) geometricamente correto para o setor"""
        if self.direcao == 1:  # A->B
            return (0.0, 0.5)  # x0 = 0 (esquerda), y0 = 0.5 (centro vertical)
        else:  # B->A
            return (1.0, 0.5)  # x0 = 1 (direita), y0 = 0.5 (centro vertical)

class Semaforo:
    """Classe que representa o semáforo como autômato híbrido separado"""
    
    def __init__(self):
        pass
    
    def calcular_sinal(self, sA, sB, prox_A, prox_B):
        """Calcula o sinal baseado nas posições atuais e próximas dos navios"""
        # Distância absoluta entre setores: |sA - sB|
        dist_maior_que_2 = Or(
            GT(Minus(sA, sB), Int(2)),  # sA - sB > 2
            GT(Minus(sB, sA), Int(2))   # sB - sA > 2
        )
        
        # Condição para VERMELHO: risco de colisão iminente
        cond_vermelho = Or(
            Equals(prox_A, sB),  # Navio A quer ir para setor de B
            Equals(prox_B, sA)   # Navio B quer ir para setor de A
        )
        
        # Condição para AMARELO: risco moderado (setores adjacentes)
        cond_amarelo = And(
            Not(cond_vermelho),  # Não é caso vermelho
            Or(
                Equals(prox_A, Plus(sB, Int(1))),
                Equals(prox_A, Minus(sB, Int(1))),
                Equals(prox_B, Plus(sA, Int(1))),
                Equals(prox_B, Minus(sA, Int(1)))
            )
        )
        
        # Condição para VERDE: distância segura (>2 setores) ou situação estável
        cond_verde = Or(
            dist_maior_que_2,  # Distância > 2 setores
            And(
                Not(cond_vermelho),
                Not(cond_amarelo)
            )
        )
        
        # Retorna expressão para o sinal
        return Ite(
            cond_vermelho,
            SINAL_VERMELHO,
            Ite(
                cond_amarelo,
                SINAL_AMARELO,
                SINAL_VERDE
            )
        )

# ============================================
# PARTE 2: FOTS GLOBAL COM PARÂMETROS POR SETOR
# ============================================

def declare_fots(i):
    """Declara as variáveis do FOTS para o estado i - COM PARÂMETROS POR SETOR"""
    s = {}
    # Setores dos navios
    s['sA'] = Symbol(f'sA{i}', INT)
    s['sB'] = Symbol(f'sB{i}', INT)
    
    # Variáveis de movimento dos navios
    s['zA'] = Symbol(f'zA{i}', REAL)  # deslocamento ao longo do rumo
    s['zB'] = Symbol(f'zB{i}', REAL)
    s['vA'] = Symbol(f'vA{i}', REAL)  # velocidade
    s['vB'] = Symbol(f'vB{i}', REAL)
    
    # Variáveis físicas dos navios (AGORA VARIAM POR SETOR)
    s['gammaA'] = Symbol(f'gammaA{i}', REAL)  # aceleração (propulsão) - depende do setor
    s['gammaB'] = Symbol(f'gammaB{i}', REAL)
    s['epsilonA'] = Symbol(f'epsilonA{i}', REAL)  # força da corrente - depende do setor
    s['epsilonB'] = Symbol(f'epsilonB{i}', REAL)
    s['V_localA'] = Symbol(f'V_localA{i}', REAL)  # limite de velocidade no setor atual
    s['V_localB'] = Symbol(f'V_localB{i}', REAL)
    
    # Ponto inicial no setor
    s['x0A'] = Symbol(f'x0A{i}', REAL)
    s['x0B'] = Symbol(f'x0B{i}', REAL)
    s['y0A'] = Symbol(f'y0A{i}', REAL)
    s['y0B'] = Symbol(f'y0B{i}', REAL)
    
    # Coordenadas atuais dos navios
    s['xA'] = Symbol(f'xA{i}', REAL)
    s['xB'] = Symbol(f'xB{i}', REAL)
    s['yA'] = Symbol(f'yA{i}', REAL)
    s['yB'] = Symbol(f'yB{i}', REAL)
    
    # Ângulo de rumo
    s['phiA'] = Symbol(f'phiA{i}', REAL)
    s['phiB'] = Symbol(f'phiB{i}', REAL)
    
    # Modos dos navios
    s['modoA'] = Symbol(f'modoA{i}', INT)
    s['modoB'] = Symbol(f'modoB{i}', INT)
    
    # Variáveis do semáforo
    s['sinal'] = Symbol(f'sinal{i}', INT)
    s['t_semaforo'] = Symbol(f't_semaforo{i}', REAL)
    s['t_global'] = Symbol(f't_global{i}', REAL)
    
    return s

def init_fots(navioA, navioB, semaforo):
    """Retorna o predicado inicial do FOTS - COM PARÂMETROS POR SETOR"""
    def init_pred(s):
        # Parâmetros iniciais dos navios (variam por setor)
        gammaA0, epsilonA0, V_localA0 = navioA.get_parametros(0)
        gammaB0, epsilonB0, V_localB0 = navioB.get_parametros(14)
        
        # Pontos iniciais geometricamente corretos
        x0A_init, y0A_init = navioA.get_ponto_inicial_geometrico(0)
        x0B_init, y0B_init = navioB.get_ponto_inicial_geometrico(14)
        
        # Calcular sinal inicial
        sinal_inicial = semaforo.calcular_sinal(
            Int(0), Int(14),
            navioA.proximo_setor_expr(Int(0)),
            navioB.proximo_setor_expr(Int(14))
        )
        
        # Rumos iniciais
        phiA_init = Real(navioA.rumo)
        phiB_init = Real(navioB.rumo)
        
        return And(
            # Navio A começa no setor 0
            Equals(s['sA'], Int(0)),
            Equals(s['zA'], Real(0.0)),
            Equals(s['vA'], Real(0.0)),
            # Parâmetros específicos do setor 0
            Equals(s['gammaA'], Real(gammaA0)),
            Equals(s['epsilonA'], Real(epsilonA0)),
            Equals(s['V_localA'], Real(V_localA0)),
            
            Equals(s['x0A'], Real(x0A_init)),
            Equals(s['y0A'], Real(y0A_init)),
            Equals(s['phiA'], phiA_init),
            Equals(s['xA'], Real(x0A_init)),
            Equals(s['yA'], Real(y0A_init)),
            Equals(s['modoA'], NAVIO_PARADO),
            
            # Navio B começa no setor 14
            Equals(s['sB'], Int(14)),
            Equals(s['zB'], Real(0.0)),
            Equals(s['vB'], Real(0.0)),
            # Parâmetros específicos do setor 14
            Equals(s['gammaB'], Real(gammaB0)),
            Equals(s['epsilonB'], Real(epsilonB0)),
            Equals(s['V_localB'], Real(V_localB0)),
            
            Equals(s['x0B'], Real(x0B_init)),
            Equals(s['y0B'], Real(y0B_init)),
            Equals(s['phiB'], phiB_init),
            Equals(s['xB'], Real(x0B_init)),
            Equals(s['yB'], Real(y0B_init)),
            Equals(s['modoB'], NAVIO_PARADO),
            
            # Semáforo
            Equals(s['sinal'], sinal_inicial),
            Equals(s['t_semaforo'], Real(0.0)),
            Equals(s['t_global'], Real(0.0)),
            
            # Restrições de domínio básicas
            GE(s['vA'], Real(0.0)), LE(s['vA'], Real(VEL_MAX)),
            GE(s['vB'], Real(0.0)), LE(s['vB'], Real(VEL_MAX)),
            
            # Setores válidos
            GE(s['sA'], Int(0)), LE(s['sA'], Int(NUM_SETORES - 1)),
            GE(s['sB'], Int(0)), LE(s['sB'], Int(NUM_SETORES - 1)),
            
            # Limites de velocidade locais
            LE(s['V_localA'], Real(VEL_MAX)),
            LE(s['V_localB'], Real(VEL_MAX)),
            GE(s['V_localA'], Real(0.0)),
            GE(s['V_localB'], Real(0.0)),
            
            # Restrições geométricas
            GE(s['xA'], Real(0.0)), LE(s['xA'], Real(1.0)),
            GE(s['yA'], Real(0.0)), LE(s['yA'], Real(1.0)),
            GE(s['xB'], Real(0.0)), LE(s['xB'], Real(1.0)),
            GE(s['yB'], Real(0.0)), LE(s['yB'], Real(1.0)),
            
            # Navios começam em setores diferentes
            Not(Equals(s['sA'], s['sB']))
        )
    return init_pred

def equacoes_movimento(p_z, p_v, z, v, gamma, epsilon, V_local, dt_value=0.1):
    """Retorna as equações de movimento para um intervalo dt - COM PARÂMETROS POR SETOR"""
    dt = Real(dt_value)
    sigma = Real(SIGMA)
    
    # Dois regimes conforme enunciado:
    # dv/dt + σv = γ se v ≤ V_local
    # dv/dt + σv = ε se v > V_local
    # dz/dt = v
    
    condicao_v_le_V = LE(v, V_local)
    
    # Equações para v ≤ V_local
    dv_dt_le = Minus(gamma, Times(sigma, v))
    v_next_le = Plus(v, Times(dv_dt_le, dt))
    z_next_le = Plus(z, Times(v, dt))
    
    # Equações para v > V_local
    dv_dt_gt = Minus(epsilon, Times(sigma, v))
    v_next_gt = Plus(v, Times(dv_dt_gt, dt))
    z_next_gt = Plus(z, Times(v, dt))
    
    return And(
        Ite(condicao_v_le_V,
            And(Equals(p_v, v_next_le), Equals(p_z, z_next_le)),
            And(Equals(p_v, v_next_gt), Equals(p_z, z_next_gt))
        ),
        GE(p_v, Real(0.0)),  # velocidade não-negativa
        LE(p_v, Real(VEL_MAX))  # limite global
    )

def restricoes_geometricas_simplificada(x, y, x0, y0, z, phi):
    """Versão simplificada das restrições geométricas para movimento horizontal"""
    # Para φ = 0: x = x0 + z, y = y0
    # Para φ = π: x = x0 - z, y = y0
    
    x_pos = Ite(Equals(phi, Real(0.0)),
                Plus(x0, z),      # φ = 0: direita
                Minus(x0, z))     # φ = π: esquerda
    
    return And(
        # Coordenadas dentro do setor
        GE(x, Real(0.0)), LE(x, Real(1.0)),
        GE(y, Real(0.0)), LE(y, Real(1.0)),
        # Relação geométrica
        Equals(x, x_pos),
        Equals(y, y0)
    )

def trans_fots(navioA, navioB, semaforo):
    """Retorna o predicado de transição do FOTS global - COM PARÂMETROS POR SETOR"""
    
    def trans_pred(s, p):
        # Extrair variáveis atuais
        sA, sB = s['sA'], s['sB']
        zA, zB = s['zA'], s['zB']
        vA, vB = s['vA'], s['vB']
        modoA, modoB = s['modoA'], s['modoB']
        sinal = s['sinal']
        t_semaforo = s['t_semaforo']
        t_global = s['t_global']
        
        gammaA, gammaB = s['gammaA'], s['gammaB']
        epsilonA, epsilonB = s['epsilonA'], s['epsilonB']
        V_localA, V_localB = s['V_localA'], s['V_localB']
        x0A, x0B = s['x0A'], s['x0B']
        y0A, y0B = s['y0A'], s['y0B']
        phiA, phiB = s['phiA'], s['phiB']
        
        # Extrair variáveis próximas
        p_sA, p_sB = p['sA'], p['sB']
        p_zA, p_zB = p['zA'], p['zB']
        p_vA, p_vB = p['vA'], p['vB']
        p_modoA, p_modoB = p['modoA'], p['modoB']
        p_sinal = p['sinal']
        p_t_semaforo = p['t_semaforo']
        p_t_global = p['t_global']
        
        p_gammaA, p_gammaB = p['gammaA'], p['gammaB']
        p_epsilonA, p_epsilonB = p['epsilonA'], p['epsilonB']
        p_V_localA, p_V_localB = p['V_localA'], p['V_localB']
        p_x0A, p_x0B = p['x0A'], p['x0B']
        p_y0A, p_y0B = p['y0A'], p['y0B']
        p_phiA, p_phiB = p['phiA'], p['phiB']
        
        # Expressões para próximos setores
        prox_A = navioA.proximo_setor_expr(sA)
        prox_B = navioB.proximo_setor_expr(sB)
        
        # Parâmetros para próximo setor (PARÂMETROS ESPECÍFICOS POR SETOR)
        gammaA_next, epsilonA_next, V_localA_next = navioA.get_parametros(prox_A)
        gammaB_next, epsilonB_next, V_localB_next = navioB.get_parametros(prox_B)
        
        # Pontos iniciais para próximo setor
        x0A_next, y0A_next = navioA.get_ponto_inicial_geometrico(prox_A)
        x0B_next, y0B_next = navioB.get_ponto_inicial_geometrico(prox_B)
        
        # Restrições básicas
        base_constraints = And(
            # Modos válidos
            Or(Equals(p_modoA, NAVIO_PARADO), Equals(p_modoA, NAVIO_MOVENDO)),
            Or(Equals(p_modoB, NAVIO_PARADO), Equals(p_modoB, NAVIO_MOVENDO)),
            
            # Sinal válido
            Or(Equals(p_sinal, SINAL_VERMELHO), 
               Equals(p_sinal, SINAL_AMARELO), 
               Equals(p_sinal, SINAL_VERDE)),
            
            # Velocidades não-negativas
            GE(p_vA, Real(0.0)), GE(p_vB, Real(0.0)),
            
            # Setores válidos
            GE(p_sA, Int(0)), LE(p_sA, Int(NUM_SETORES - 1)),
            GE(p_sB, Int(0)), LE(p_sB, Int(NUM_SETORES - 1)),
            
            # Coordenadas dentro do setor
            GE(p['xA'], Real(0.0)), LE(p['xA'], Real(1.0)),
            GE(p['yA'], Real(0.0)), LE(p['yA'], Real(1.0)),
            GE(p['xB'], Real(0.0)), LE(p['xB'], Real(1.0)),
            GE(p['yB'], Real(0.0)), LE(p['yB'], Real(1.0))
        )
        
        # 1. Transição Timed - evolução contínua COM PARÂMETROS DO SETOR ATUAL
        timed_trans = And(
            base_constraints,
            # Tempo avança
            GT(p_t_global, t_global),
            GT(p_t_semaforo, t_semaforo),
            LT(Minus(p_t_global, t_global), Real(1.0)),
            
            # Setores não mudam
            Equals(p_sA, sA),
            Equals(p_sB, sB),
            
            # Semáforo: modos mantêm, sinal mantêm
            Equals(p_sinal, sinal),
            
            # Parâmetros físicos não mudam (dentro do mesmo setor)
            Equals(p_gammaA, gammaA), Equals(p_gammaB, gammaB),
            Equals(p_epsilonA, epsilonA), Equals(p_epsilonB, epsilonB),
            Equals(p_V_localA, V_localA), Equals(p_V_localB, V_localB),
            Equals(p_x0A, x0A), Equals(p_x0B, x0B),
            Equals(p_y0A, y0A), Equals(p_y0B, y0B),
            Equals(p_phiA, phiA), Equals(p_phiB, phiB),
            
            # Navio A: evolução conforme modo COM PARÂMETROS DO SETOR ATUAL
            Ite(
                Equals(modoA, NAVIO_MOVENDO),
                And(
                    # Equações de movimento com parâmetros do setor atual
                    equacoes_movimento(p_zA, p_vA, zA, vA, gammaA, epsilonA, V_localA, 0.1),
                    # Verificação geométrica
                    restricoes_geometricas_simplificada(p['xA'], p['yA'], x0A, y0A, p_zA, phiA),
                    Equals(p_modoA, NAVIO_MOVENDO)
                ),
                # Parado: mantém valores
                And(
                    Equals(p_zA, zA),
                    Equals(p_vA, vA),
                    Equals(p['xA'], s['xA']),
                    Equals(p['yA'], s['yA']),
                    Equals(p_modoA, modoA)
                )
            ),
            
            # Navio B similar
            Ite(
                Equals(modoB, NAVIO_MOVENDO),
                And(
                    equacoes_movimento(p_zB, p_vB, zB, vB, gammaB, epsilonB, V_localB, 0.1),
                    restricoes_geometricas_simplificada(p['xB'], p['yB'], x0B, y0B, p_zB, phiB),
                    Equals(p_modoB, NAVIO_MOVENDO)
                ),
                And(
                    Equals(p_zB, zB),
                    Equals(p_vB, vB),
                    Equals(p['xB'], s['xB']),
                    Equals(p['yB'], s['yB']),
                    Equals(p_modoB, modoB)
                )
            )
        )
        
        # 2. Transição Untimed: Navio A inicia movimento
        start_A = And(
            base_constraints,
            # Pré-condições
            Equals(modoA, NAVIO_PARADO),
            Equals(sinal, SINAL_VERDE),
            
            # Ações do navio
            Equals(p_modoA, NAVIO_MOVENDO),
            Equals(p_vA, Real(0.5)),  # velocidade inicial baixa
            Equals(p_zA, zA),
            
            # Verificação geométrica
            restricoes_geometricas_simplificada(p['xA'], p['yA'], x0A, y0A, p_zA, phiA),
            
            # Semáforo: sincronização
            Equals(p_t_semaforo, t_semaforo),
            Equals(p_t_global, t_global),
            Equals(p_sinal, semaforo.calcular_sinal(sA, sB, prox_A, prox_B)),
            
            # Parâmetros mantêm (mesmo setor)
            Equals(p_sA, sA), Equals(p_sB, sB),
            Equals(p_zB, zB), Equals(p_vB, vB), Equals(p_modoB, modoB),
            Equals(p_gammaA, gammaA), Equals(p_gammaB, gammaB),
            Equals(p_epsilonA, epsilonA), Equals(p_epsilonB, epsilonB),
            Equals(p_V_localA, V_localA), Equals(p_V_localB, V_localB),
            Equals(p_x0A, x0A), Equals(p_x0B, x0B),
            Equals(p_y0A, y0A), Equals(p_y0B, y0B),
            Equals(p_phiA, phiA), Equals(p_phiB, phiB),
            # Navio B mantém geometria
            restricoes_geometricas_simplificada(p['xB'], p['yB'], x0B, y0B, p_zB, phiB)
        )
        
        # 3. Transição Untimed: Navio A muda de setor
        transition_A = And(
            base_constraints,
            # Pré-condições
            Equals(modoA, NAVIO_MOVENDO),
            GE(zA, Real(LADO_SETOR)),  # completou o setor
            
            # Condições de transição dependem do sinal
            Or(
                # Sinal VERDE e próximo setor não ocupado
                And(
                    Equals(sinal, SINAL_VERDE),
                    Not(Equals(prox_A, sB)),
                    # Transição permitida
                    And(
                        Equals(p_sA, prox_A),
                        Equals(p_zA, Real(0.0)),  # reinicia deslocamento
                        Equals(p_vA, vA),  # mantém velocidade
                        Equals(p_modoA, NAVIO_MOVENDO),
                        # ATUALIZA PARÂMETROS PARA O NOVO SETOR
                        Equals(p_gammaA, Real(gammaA_next)),
                        Equals(p_epsilonA, Real(epsilonA_next)),
                        Equals(p_V_localA, Real(V_localA_next)),
                        # Novo ponto inicial
                        Equals(p_x0A, Real(x0A_next)),
                        Equals(p_y0A, Real(y0A_next)),
                        # Rumo mantém
                        Equals(p_phiA, phiA),
                        # Verifica geometria no novo setor
                        restricoes_geometricas_simplificada(p['xA'], p['yA'], 
                                                          Real(x0A_next), Real(y0A_next), 
                                                          Real(0.0), phiA)
                    )
                ),
                # Sinal AMARELO, próximo setor não ocupado e não adjacente
                And(
                    Equals(sinal, SINAL_AMARELO),
                    Not(Equals(prox_A, sB)),
                    Not(Or(
                        Equals(prox_A, Plus(sB, Int(1))),
                        Equals(prox_A, Minus(sB, Int(1)))
                    )),
                    And(
                        Equals(p_sA, prox_A),
                        Equals(p_zA, Real(0.0)),
                        Equals(p_vA, vA),
                        Equals(p_modoA, NAVIO_MOVENDO),
                        Equals(p_gammaA, Real(gammaA_next)),
                        Equals(p_epsilonA, Real(epsilonA_next)),
                        Equals(p_V_localA, Real(V_localA_next)),
                        Equals(p_x0A, Real(x0A_next)),
                        Equals(p_y0A, Real(y0A_next)),
                        Equals(p_phiA, phiA),
                        restricoes_geometricas_simplificada(p['xA'], p['yA'], 
                                                          Real(x0A_next), Real(y0A_next), 
                                                          Real(0.0), phiA)
                    )
                ),
                # Não pode transitar - para
                And(
                    Or(
                        Equals(sinal, SINAL_VERMELHO),
                        Equals(prox_A, sB),
                        And(
                            Equals(sinal, SINAL_AMARELO),
                            Or(
                                Equals(prox_A, Plus(sB, Int(1))),
                                Equals(prox_A, Minus(sB, Int(1)))
                            )
                        )
                    ),
                    # Fica parado no mesmo setor
                    And(
                        Equals(p_sA, sA),
                        Equals(p_zA, Real(0.0)),
                        Equals(p_vA, Real(0.0)),
                        Equals(p_modoA, NAVIO_PARADO),
                        # Mantém parâmetros do setor atual
                        Equals(p_gammaA, gammaA),
                        Equals(p_epsilonA, epsilonA),
                        Equals(p_V_localA, V_localA),
                        Equals(p_x0A, x0A),
                        Equals(p_y0A, y0A),
                        Equals(p_phiA, phiA),
                        # Geometria: volta ao ponto inicial
                        restricoes_geometricas_simplificada(p['xA'], p['yA'], x0A, y0A, Real(0.0), phiA)
                    )
                )
            ),
            
            # Semáforo: sincronização
            Equals(p_t_semaforo, t_semaforo),
            Equals(p_t_global, t_global),
            Equals(p_sinal, semaforo.calcular_sinal(p_sA, sB,
                                                   navioA.proximo_setor_expr(p_sA),
                                                   prox_B)),
            
            # Demais variáveis inalteradas (navio B)
            Equals(p_sB, sB),
            Equals(p_zB, zB), Equals(p_vB, vB), Equals(p_modoB, modoB),
            Equals(p_gammaB, gammaB), Equals(p_epsilonB, epsilonB),
            Equals(p_V_localB, V_localB),
            Equals(p_x0B, x0B), Equals(p_y0B, y0B),
            Equals(p_phiB, phiB),
            # Navio B mantém geometria
            restricoes_geometricas_simplificada(p['xB'], p['yB'], x0B, y0B, p_zB, phiB)
        )
        
        # 4. Transições similares para navio B
        start_B = And(
            base_constraints,
            Equals(modoB, NAVIO_PARADO),
            Equals(sinal, SINAL_VERDE),
            Equals(p_modoB, NAVIO_MOVENDO),
            Equals(p_vB, Real(0.5)),
            Equals(p_zB, zB),
            restricoes_geometricas_simplificada(p['xB'], p['yB'], x0B, y0B, p_zB, phiB),
            
            # Semáforo
            Equals(p_t_semaforo, t_semaforo),
            Equals(p_t_global, t_global),
            Equals(p_sinal, semaforo.calcular_sinal(sA, sB, prox_A, prox_B)),
            
            # Demais inalteradas
            Equals(p_sA, sA), Equals(p_zA, zA), Equals(p_vA, vA), Equals(p_modoA, modoA),
            Equals(p_sB, sB),
            Equals(p_gammaA, gammaA), Equals(p_gammaB, gammaB),
            Equals(p_epsilonA, epsilonA), Equals(p_epsilonB, epsilonB),
            Equals(p_V_localA, V_localA), Equals(p_V_localB, V_localB),
            Equals(p_x0A, x0A), Equals(p_x0B, x0B),
            Equals(p_y0A, y0A), Equals(p_y0B, y0B),
            Equals(p_phiA, phiA), Equals(p_phiB, phiB),
            # Navio A mantém geometria
            restricoes_geometricas_simplificada(p['xA'], p['yA'], x0A, y0A, p_zA, phiA)
        )
        
        transition_B = And(
            base_constraints,
            Equals(modoB, NAVIO_MOVENDO),
            GE(zB, Real(LADO_SETOR)),
            Or(
                And(
                    Equals(sinal, SINAL_VERDE),
                    Not(Equals(prox_B, sA)),
                    And(
                        Equals(p_sB, prox_B),
                        Equals(p_zB, Real(0.0)),
                        Equals(p_vB, vB),
                        Equals(p_modoB, NAVIO_MOVENDO),
                        # ATUALIZA PARÂMETROS PARA O NOVO SETOR
                        Equals(p_gammaB, Real(gammaB_next)),
                        Equals(p_epsilonB, Real(epsilonB_next)),
                        Equals(p_V_localB, Real(V_localB_next)),
                        Equals(p_x0B, Real(x0B_next)),
                        Equals(p_y0B, Real(y0B_next)),
                        Equals(p_phiB, phiB),
                        restricoes_geometricas_simplificada(p['xB'], p['yB'], 
                                                          Real(x0B_next), Real(y0B_next), 
                                                          Real(0.0), phiB)
                    )
                ),
                And(
                    Equals(sinal, SINAL_AMARELO),
                    Not(Equals(prox_B, sA)),
                    Not(Or(
                        Equals(prox_B, Plus(sA, Int(1))),
                        Equals(prox_B, Minus(sA, Int(1)))
                    )),
                    And(
                        Equals(p_sB, prox_B),
                        Equals(p_zB, Real(0.0)),
                        Equals(p_vB, vB),
                        Equals(p_modoB, NAVIO_MOVENDO),
                        Equals(p_gammaB, Real(gammaB_next)),
                        Equals(p_epsilonB, Real(epsilonB_next)),
                        Equals(p_V_localB, Real(V_localB_next)),
                        Equals(p_x0B, Real(x0B_next)),
                        Equals(p_y0B, Real(y0B_next)),
                        Equals(p_phiB, phiB),
                        restricoes_geometricas_simplificada(p['xB'], p['yB'], 
                                                          Real(x0B_next), Real(y0B_next), 
                                                          Real(0.0), phiB)
                    )
                ),
                And(
                    Or(
                        Equals(sinal, SINAL_VERMELHO),
                        Equals(prox_B, sA),
                        And(
                            Equals(sinal, SINAL_AMARELO),
                            Or(
                                Equals(prox_B, Plus(sA, Int(1))),
                                Equals(prox_B, Minus(sA, Int(1)))
                            )
                        )
                    ),
                    And(
                        Equals(p_sB, sB),
                        Equals(p_zB, Real(0.0)),
                        Equals(p_vB, Real(0.0)),
                        Equals(p_modoB, NAVIO_PARADO),
                        Equals(p_gammaB, gammaB),
                        Equals(p_epsilonB, epsilonB),
                        Equals(p_V_localB, V_localB),
                        Equals(p_x0B, x0B),
                        Equals(p_y0B, y0B),
                        Equals(p_phiB, phiB),
                        restricoes_geometricas_simplificada(p['xB'], p['yB'], x0B, y0B, Real(0.0), phiB)
                    )
                )
            ),
            
            # Semáforo
            Equals(p_t_semaforo, t_semaforo),
            Equals(p_t_global, t_global),
            Equals(p_sinal, semaforo.calcular_sinal(
                sA, p_sB,
                prox_A,
                navioB.proximo_setor_expr(p_sB)
            )),
            
            # Demais inalteradas
            Equals(p_sA, sA), Equals(p_zA, zA), Equals(p_vA, vA), Equals(p_modoA, modoA),
            Equals(p_gammaA, gammaA), Equals(p_epsilonA, epsilonA),
            Equals(p_V_localA, V_localA),
            Equals(p_x0A, x0A), Equals(p_y0A, y0A),
            Equals(p_phiA, phiA),
            # Navio A mantém geometria
            restricoes_geometricas_simplificada(p['xA'], p['yA'], x0A, y0A, p_zA, phiA)
        )
        
        # 5. Transição independente do semáforo
        update_semaforo = And(
            base_constraints,
            # Semáforo atualiza periodicamente
            Equals(p_t_semaforo, Plus(t_semaforo, Real(0.1))),
            Equals(p_t_global, t_global),
            
            # Atualiza sinal
            Equals(p_sinal, semaforo.calcular_sinal(sA, sB, prox_A, prox_B)),
            
            # Navios não se movem
            Equals(p_sA, sA), Equals(p_sB, sB),
            Equals(p_zA, zA), Equals(p_zB, zB),
            Equals(p_vA, vA), Equals(p_vB, vB),
            Equals(p_modoA, modoA), Equals(p_modoB, modoB),
            Equals(p_gammaA, gammaA), Equals(p_gammaB, gammaB),
            Equals(p_epsilonA, epsilonA), Equals(p_epsilonB, epsilonB),
            Equals(p_V_localA, V_localA), Equals(p_V_localB, V_localB),
            Equals(p_x0A, x0A), Equals(p_x0B, x0B),
            Equals(p_y0A, y0A), Equals(p_y0B, y0B),
            Equals(p_phiA, phiA), Equals(p_phiB, phiB),
            # Geometria mantém
            restricoes_geometricas_simplificada(p['xA'], p['yA'], x0A, y0A, p_zA, phiA),
            restricoes_geometricas_simplificada(p['xB'], p['yB'], x0B, y0B, p_zB, phiB)
        )
        
        return Or(timed_trans, start_A, transition_A, start_B, transition_B, update_semaforo)
    
    return trans_pred

# ============================================
# PARTE 3: VERIFICAÇÃO DE SEGURANÇA
# ============================================

def safety_sufficient(s):
    """Condição de segurança suficiente: navios não podem estar no mesmo setor"""
    return Not(Equals(s['sA'], s['sB']))

def create_safety_strong_predicate(navioA, navioB):
    """Cria predicado de segurança forte"""
    def safety_strong(s):
        prox_A = navioA.proximo_setor_expr(s['sA'])
        prox_B = navioB.proximo_setor_expr(s['sB'])
        
        navioA_bloqueado = And(
            Equals(s['modoA'], NAVIO_PARADO),
            Or(
                Equals(prox_A, s['sB']),
                Equals(s['sinal'], SINAL_VERMELHO),
                And(
                    Equals(s['sinal'], SINAL_AMARELO),
                    Or(
                        Equals(prox_A, Plus(s['sB'], Int(1))),
                        Equals(prox_A, Minus(s['sB'], Int(1)))
                    )
                )
            )
        )
        
        navioB_bloqueado = And(
            Equals(s['modoB'], NAVIO_PARADO),
            Or(
                Equals(prox_B, s['sA']),
                Equals(s['sinal'], SINAL_VERMELHO),
                And(
                    Equals(s['sinal'], SINAL_AMARELO),
                    Or(
                        Equals(prox_B, Plus(s['sA'], Int(1))),
                        Equals(prox_B, Minus(s['sA'], Int(1)))
                    )
                )
            )
        )
        
        return And(
            safety_sufficient(s),
            Not(And(navioA_bloqueado, navioB_bloqueado))
        )
    return safety_strong

# ============================================
# FUNÇÕES DE VERIFICAÇÃO
# ============================================

def bmc_always(declare, init, trans, prop, K):
    """Bounded Model Checking para propriedade G(prop)"""
    for k in range(1, K+1):
        with Solver(name="z3") as solver:
            estados = [declare(i) for i in range(k+1)]
            
            solver.add_assertion(init(estados[0]))
            for i in range(k):
                solver.add_assertion(trans(estados[i], estados[i+1]))
            
            not_prop = Or([Not(prop(estados[i])) for i in range(k+1)])
            solver.add_assertion(not_prop)
            
            if solver.solve():
                print(f"Contra-exemplo encontrado com {k} passos:")
                model = solver.get_model()
                
                for i, estado in enumerate(estados):
                    print(f"\nEstado {i}:")
                    for nome, var in estado.items():
                        try:
                            val = model.get_py_value(var)
                            if var.symbol_type().is_real_type():
                                val = float(val)
                                print(f"  {nome} = {val:.3f}")
                            else:
                                print(f"  {nome} = {val}")
                        except:
                            print(f"  {nome} = (valor não disponível)")
                
                return False
    
    print(f"Propriedade válida para traços de até {K} passos")
    return True

def k_induction(declare, init, trans, prop, k):
    """Verificação por k-indução"""
    
    # Caso base
    with Solver(name="z3") as solver:
        estados = [declare(i) for i in range(k+1)]
        
        traco = [init(estados[0])]
        for i in range(k):
            traco.append(trans(estados[i], estados[i+1]))
        
        violacao = Or([Not(prop(estados[i])) for i in range(k+1)])
        
        solver.add_assertion(And(And(traco), violacao))
        if solver.solve():
            print("Caso base falha")
            model = solver.get_model()
            for i, estado in enumerate(estados[:k+1]):
                print(f"\nEstado {i}:")
                for nome, var in estado.items():
                    try:
                        val = model.get_py_value(var)
                        if var.symbol_type().is_real_type():
                            val = float(val)
                            print(f"  {nome} = {val:.3f}")
                        else:
                            print(f"  {nome} = {val}")
                    except:
                        print(f"  {nome} = (valor não disponível)")
            return False
    
    # Passo indutivo
    with Solver(name="z3") as solver:
        estados = [declare(i) for i in range(k+2)]
        
        hipotese = And([prop(estados[i]) for i in range(k+1)])
        transicoes = [trans(estados[i], estados[i+1]) for i in range(k+1)]
        
        solver.add_assertion(And(And(transicoes), hipotese, Not(prop(estados[k+1]))))
        
        if solver.solve():
            print("Passo indutivo falha")
            return False
    
    print(f"Propriedade verificada por {k}-indução")
    return True

# ============================================
# EXECUÇÃO PRINCIPAL
# ============================================

def main():
    print("=== TRABALHO PRÁTICO 4: CONTROLE DE TRÁFEGO MARÍTIMO ===\n")
    print("VERSÃO COMPLETA COM DIFERENCIAÇÃO POR SETOR NOS PARÂMETROS\n")
    
    # 1. Criar os autômatos híbridos
    print("1. Criando autômatos híbridos com parâmetros por setor...")
    
    navioA = Navio("Navio A->B", 0, 1)
    navioB = Navio("Navio B->A", 14, -1)
    semaforo = Semaforo()
    
    print(f"   Navio A: inicia no setor 0, direção A->B, rumo={navioA.rumo:.2f} rad")
    print(f"   Navio B: inicia no setor 14, direção B->A, rumo={navioB.rumo:.2f} rad")
    print("   Sistema com diferenciação completa por setor nos parâmetros físicos")
    
    # 2. Construir FOTS global
    print("\n2. Construindo FOTS global com parâmetros por setor...")
    
    init = init_fots(navioA, navioB, semaforo)
    trans = trans_fots(navioA, navioB, semaforo)
    
    print("   FOTS construído com sucesso")
    print(f"   Variáveis de estado: {list(declare_fots(0).keys())}")
    
    # 3. Verificar segurança
    print("\n3. Verificando segurança do sistema...")
    
    print("\n   a) Segurança suficiente (navios nunca no mesmo setor):")
    print("   Usando BMC com K=3...")
    resultado_suficiente = bmc_always(declare_fots, init, trans, safety_sufficient, 3)
    
    print("\n   Usando 2-indução...")
    resultado_suficiente_ind = k_induction(declare_fots, init, trans, safety_sufficient, 2)
    
    print("\n   b) Segurança forte (sem deadlock):")
    print("   Usando BMC com K=3...")
    safety_strong_pred = create_safety_strong_predicate(navioA, navioB)
    resultado_forte = bmc_always(declare_fots, init, trans, safety_strong_pred, 3)
    
    print("\n   Usando 2-indução...")
    resultado_forte_ind = k_induction(declare_fots, init, trans, safety_strong_pred, 2)
    
    # 4. Resumo dos resultados
    print("\n=== RESUMO DOS RESULTADOS ===")
    print(f"Segurança suficiente (BMC): {'VÁLIDA' if resultado_suficiente else 'INVÁLIDA'}")
    print(f"Segurança suficiente (k-indução): {'VÁLIDA' if resultado_suficiente_ind else 'INVÁLIDA'}")
    print(f"Segurança forte (BMC): {'VÁLIDA' if resultado_forte else 'INVÁLIDA'}")
    print(f"Segurança forte (k-indução): {'VÁLIDA' if resultado_forte_ind else 'INVÁLIDA'}")
    
    # 5. Interpretação
    print("\n=== INTERPRETAÇÃO DOS RESULTADOS ===")
    if not resultado_suficiente:
        print("O sistema NÃO é seguro na versão básica.")
        print("Possíveis causas:")
        print("1. Parâmetros diferentes por setor podem criar situações perigosas")
        print("2. Acelerações/desacelerações podem causar encontros inesperados")
    
    if not resultado_forte:
        print("\nO sistema também NÃO é seguro na versão forte.")
        print("Deadlock possível mesmo com parâmetros por setor.")
        print("\nRecomendações:")
        print("1. Revisar parâmetros de aceleração/desaceleração por setor")
        print("2. Ajustar a lógica do semáforo considerando as diferentes dinâmicas")
        print("3. Implementar zonas de espera mais largas em setores críticos")

if __name__ == "__main__":
    main()

=== TRABALHO PRÁTICO 4: CONTROLE DE TRÁFEGO MARÍTIMO ===

VERSÃO COMPLETA COM DIFERENCIAÇÃO POR SETOR NOS PARÂMETROS

1. Criando autômatos híbridos com parâmetros por setor...
   Navio A: inicia no setor 0, direção A->B, rumo=0.00 rad
   Navio B: inicia no setor 14, direção B->A, rumo=3.14 rad
   Sistema com diferenciação completa por setor nos parâmetros físicos

2. Construindo FOTS global com parâmetros por setor...
   FOTS construído com sucesso
   Variáveis de estado: ['sA', 'sB', 'zA', 'zB', 'vA', 'vB', 'gammaA', 'gammaB', 'epsilonA', 'epsilonB', 'V_localA', 'V_localB', 'x0A', 'x0B', 'y0A', 'y0B', 'xA', 'xB', 'yA', 'yB', 'phiA', 'phiB', 'modoA', 'modoB', 'sinal', 't_semaforo', 't_global']

3. Verificando segurança do sistema...

   a) Segurança suficiente (navios nunca no mesmo setor):
   Usando BMC com K=3...
Propriedade válida para traços de até 3 passos

   Usando 2-indução...
Propriedade verificada por 2-indução

   b) Segurança forte (sem deadlock):
   Usando BMC com K=3...
P